In [0]:
# Define the file location and type
pin_location = "/mnt/0affc011d3cf-mount/topics/0affc011d3cf.pin/partition=0/*.json"
file_type = "json"
infer_schema = "true"

# Load the JSON files into a Spark DataFrame
df_pin = spark.read.format(file_type).option("inferSchema", infer_schema).load(pin_location)
  

In [0]:
from pyspark.sql.functions import col, when, regexp_replace

column_and_string = [
    ("description", "No description available$"),
    ("follower_count", "User Info Error"),
    ("image_src", "Image src error."),
    ("poster_name", "User Info Error"),
    ("tag_list", "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e"),
    ("title", "No Title Data Available")
]
def clean_non_relevant(df, column_and_string):
    for column, string in column_and_string:
        df = df.withColumn(column, regexp_replace(col(column), string, "None"))
    return df

def clean_empty_string(df):
    df = df.select([when(col(c)=="",None).otherwise(col(c)).alias(c) for c in df.columns])
    return df.na.fill('None')

def clean_follower_count(df):
    df = df.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
    df = df.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))
    return df.withColumn("follower_count", df["follower_count"].cast("integer"))

def clean_save_location(df):
    return df.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

def rename_index(df):
    return df.withColumnRenamed("index", "ind")

def order_columns(df):
    return df.select('ind', 'unique_id', 'title', 'description', 'follower_count', 'poster_name', 'tag_list', 'is_image_or_video', 'image_src', 'save_location', 'category')
  

In [0]:

df_pin = clean_non_relevant(df_pin, column_and_string)
df_pin = clean_empty_string(df_pin)
df_pin = clean_follower_count(df_pin)
df_pin = clean_empty_string(df_pin)
df_pin = clean_save_location(df_pin)
df_pin = rename_index(df_pin)
cleaned_pin = order_columns(df_pin)

cleaned_pin.limit(10).display()

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
7491,0bfd8ee2-8bc8-4d43-8a43-3f9e5f9678bf,ジョガーパンツでメンズコーデの足元を軽快にこなす！ | メンズファッションメディア OTOKOMAE,スポーツミックススタイルやアスレジャースタイルの台頭によって、すっかりおなじみとなった「ジョガーパンツ」。スニーカーと相性抜群なアイテムであり、フーディを合わせたカジュアルなスタイルからジャケットを羽織ったドレスライクなスタイルまで幅広くフィットする。今回はそんな「ジョガーパンツ」にフォーカスして注目の着こなし&アイテムを紹介！,122000,OTOKOMAE/男前研究所,"Streetwear,Mens Casual Hats,Fashion Week Hommes,Herren Style,Moda Blog,Look Man,La Mode Masculine,Outfits With Converse,Black Converse",image,https://i.pinimg.com/originals/5d/31/e4/5d31e49fada653798f7c8f4c47f65d14.jpg,/data/mens-fashion,mens-fashion
4362,3d2ca512-9a37-4bb5-93bc-fca20aaafdce,誕生日に親友を必ず笑顔にする11のサプライズ・アイデア♡,もうすぐ親友の誕生日。何年も共に過ごしてきただけに、毎年の誕生祝いでほとんどのアイデアは出し尽くしてしまい悩み中、、、という方に。今年はちょっと変わったバースデイサプライズで、大好きな親友の素敵な笑顔を引き出してみませんか。,256000,LOCARI,"Love Ballons,Balloons,Wedding Proposals,Marriage Proposals,Wedding Tags,Wedding Gifts,Wedding Ideas,Diy Wedding,Proposition Romantique",image,https://i.pinimg.com/originals/c8/d7/e6/c8d7e693a171205650a1ce50036c45d4.jpg,/data/event-planning,event-planning
10981,682f6f97-33d6-4b5b-bae7-b94615ea3091,"Пинтерест точно лучше инстаграмма.. | Траффик наше все! Продвижение, SEO, SMM","Пинтерест точно лучше инстаграмма #traffic #траффикнашевсе #траффик_наше_все Траффик наше все! Продвижение, SEO, SMM 👍🏻Я влюблен в эту программ",283,Edward Kramli,"Luxury Sports Cars,Top Luxury Cars,Sport Cars,Exotic Sports Cars,Mercedes Benz Amg,Carros Mercedes Benz,Benz Car,Carros Lamborghini,Carros Audi",image,https://i.pinimg.com/originals/84/a3/17/84a317acbdd5fca1c3b80ea30d45c370.jpg,/data/vehicles,vehicles
2244,20485f66-1504-4272-bb96-049d7564abdd,Christmas String Lights with Pine Cone Red Pearl Bell Garland with Lights 20 LED Warm White Battery Operated Christmas Tree Decor Light for Christmas Holiday Indoor Table Party Decoration,"6.5Ft + Pearl & Pine Cone Design - This Pearl & Pine Cone string light as a Christmas decor light. 6.5 Foot/2 Meters plated copper wire and 20 LED Lights, The led string lights…",5000,Wear24-7,"Mini Christmas Tree Decorations,Christmas Pine Cones,Christmas Fairy Lights,Led Fairy Lights,Xmas Lights,Xmas Wreaths,Christmas Bells,Outdoor Christmas,Christmas Holidays",image,https://i.pinimg.com/originals/3f/94/40/3f94400a8d9878d76db9d68126161b13.jpg,/data/christmas,christmas
2482,08604f20-fa17-4b9a-9949-781717eca6cd,FORNT PORCH CHRISTMAS DECORATING IDEAS,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",46000,"Life on Summerhill | Home, Holiday Decor & DIY Website","Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",video,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,/data/christmas,christmas
2205,24cbe878-d95a-4902-baef-f831fcb9bbd1,Toddler Christmas activity using sticky contact paper - Laughing Kids Learn,Toddler Christmas activity using sticky contact paper. Great for developing fine motor skills and more. #kbn #ohchristmastree #stickycontactpaper #handmadetree #kidschristmas #c…,99000,Laughing Kids Learn | Play Ideas for Kids | Crafts + Activities for Teachers + Parents,"Christmas Activities For Toddlers,Preschool Christmas,Holiday Activities,Christmas Paper,Craft Activities,Preschool Crafts,Christmas Themes,Christmas Decorations,Toddler Christmas Crafts",image,https://i.pinimg.com/originals/a6/61/c8/a661c87eee093cf82d3697b05b9aaee3.jpg,/data/christmas,christmas
8554,7a53684d-5311-4fa2-b25e-23a6c855fa9f,50+ Chic & Sexy Hip Tattoos for Women,beautiful Dragon ❌ flowers hip tattoo ✿ Strelkov Mikhail ®️ ✧ @strelkovtattoo ✧ Moscow 🇷🇺 ✿ 💖🐉ꕥ💖🐉ꕥ💖🐉ꕥ💖🐉ꕥ💖🐉ꕥ💖 #hiptattoo #floraltattoo #awesomeness #girltattoos #bes…,202000,kickassthin

In [0]:
# Define the Parquet file save location
parquet_save_location = "/mnt/0affc011d3cf-mount/cleaned_pin"

# Save the cleaned DataFrame as a Parquet file
cleaned_pin.write.mode('overwrite').parquet(parquet_save_location)